In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging

In [2]:
# Configuração básica de logging para acompanhar a execução
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
def extract_data(url: str) -> pd.DataFrame:
  """
  Extrai a tabela de estatísticas do VLR.gg e a retorna como um DataFrame bruto.
  """
  logging.info(f"Iniciando a extração de dados da URL: {url}")
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
  }

  try:
      response = requests.get(url, headers=headers)
      response.raise_for_status()
  except requests.exceptions.RequestException as e:
      logging.error(f"Erro ao acessar a URL: {e}")
      return None

  soup = BeautifulSoup(response.content, 'html.parser')
  stats_table = soup.find('table', class_='wf-table mod-stats mod-scroll')

  if not stats_table:
      logging.warning("Nenhuma tabela de estatísticas encontrada na página.")
      return None

  table_headers = [header.get_text(strip=True) for header in stats_table.find_all('th')]
  table_headers.pop(0)
  table_headers.insert(0, 'Player')
  table_headers.insert(1, 'Team')

  rows = []
  for row in stats_table.find('tbody').find_all('tr'):
      player_cell = row.find('td', class_='mod-player')
      player_name_div = player_cell.find('div', class_='text-of')
      player_team_div = player_cell.find('div', class_='stats-player-country')

      player = player_name_div.get_text(strip=True) if player_name_div else ''
      team = player_team_div.get_text(strip=True) if player_team_div else ''

      row_data = [player, team]
      other_cells = row.find_all('td')[1:]
      for cell in other_cells:
          span_val = cell.find('span', class_='stats-sq')
          row_data.append(span_val.get_text(strip=True) if span_val else cell.get_text(strip=True))
      rows.append(row_data)

  logging.info(f"Extração concluída. {len(rows)} linhas encontradas.")
  return pd.DataFrame(rows, columns=table_headers)

In [4]:
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
  """
  Limpa e transforma o DataFrame bruto, convertendo tipos de dados.
  """
  if df is None:
      logging.warning("DataFrame de entrada está vazio. Pulando a transformação.")
      return None

  logging.info("Iniciando a transformação dos dados.")

  # Faz uma cópia para evitar avisos de SettingWithCopyWarning
  df_transformed = df.copy()

  df_transformed = df_transformed.drop(columns=['Agents'])

  # Limpar a coluna KAST
  if 'KAST' in df_transformed.columns:
      df_transformed['KAST'] = df_transformed['KAST'].str.replace('%', '', regex=False)

  # Colunas para converter para tipo numérico
  cols_to_convert = ['R', 'ACS', 'K:D', 'ADR', 'KPR', 'APR', 'FKPR', 'FDPR', 'K', 'D', 'A', 'FK', 'FD', '+/-', 'KAST', 'KMax', 'Rnd', 'R2.0']

  for col in cols_to_convert:
      if col in df_transformed.columns:
          df_transformed[col] = pd.to_numeric(df_transformed[col], errors='coerce')

  # Converte KAST para decimal após garantir que é numérico
  if 'KAST' in df_transformed.columns:
      df_transformed['KAST'] /= 100.0

  if 'HS%' in df_transformed.columns:
        df_transformed['HS%'] = df_transformed['HS%'].str.replace('%', '').astype(float) / 100.0

  if 'CL%' in df_transformed.columns:
        df_transformed.loc[df_transformed['CL%'] == '', 'CL%'] = '0%'
        df_transformed['CL%'] = df_transformed['CL%'].str.replace('%', '').astype(float) / 100.0

  if 'CL' in df_transformed.columns:
        df_transformed.loc[df_transformed['CL'] == '', 'CL'] = '0/0'

  logging.info("Transformação concluída.")
  return df_transformed

In [5]:
def load_data(df: pd.DataFrame, file_path: str):
  """
  Salva o DataFrame final em um arquivo CSV.
  """
  if df is None:
      logging.error("DataFrame final está vazio. Nenhum dado para carregar.")
      return

  logging.info(f"Carregando dados para o arquivo: {file_path}")
  try:
      df.to_csv(file_path, index=False, encoding='utf-8-sig')
      logging.info("Dados carregados com sucesso!")
  except Exception as e:
      logging.error(f"Falha ao salvar o arquivo CSV: {e}")

In [6]:
URL = "https://www.vlr.gg/event/stats/2283/valorant-champions-2025"
OUTPUT_FILE = "valorant_stats_pipeline.csv"

In [7]:
# 1. Extract
raw_df = extract_data(URL)

In [8]:
# 2. Transform
clean_df = transform_data(raw_df)

In [9]:
# 3. Load
load_data(clean_df, OUTPUT_FILE)